# Fancy but Useful Tensor Operations

In [2]:
import torch
import numpy as np

## 基础语法

### 创建和访问

In [8]:
# 从 Python 列表创建
a = torch.tensor([1, 2, 3])
print(a)

tensor([1, 2, 3])


In [9]:
# 从 numpy.ndarray 创建
b = np.array([[1, 2, 3], [4, 5, 6]])
b = torch.tensor(b)
print(b)

tensor([[1, 2, 3],
        [4, 5, 6]])
